In [1]:
import csv
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

DATADIR = r"E:\IDM downloads\Documents\Uni\SIT University\FYP\ALLaaft\aaft70"
CATEGORIES = ['1', '2', '3']

In [2]:
sequences = list()

def create_sequences():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  # create path to categories
        class_num = CATEGORIES.index(category)  # get the classification 

        for csv in tqdm(os.listdir(path)):  # iterate over each csv file per category
            try:
                traindata = pd.read_csv(os.path.join(path,csv), header=None, names=['x', 'y', 'z'])
                values = traindata.values
                sequences.append((values,class_num))  # add this to sequences
                
            except Exception as e: 
                pass
            except OSError as e:
                print("OSError Bad csv most likely", e, os.path.join(path,csv))
            except Exception as e:
                print("general exception", e, os.path.join(path,csv))

create_sequences()

print(len(sequences))

100%|██████████| 2167/2167 [00:02<00:00, 860.67it/s]


7341


In [3]:
import random
random.shuffle(sequences)
x = []
y = []

for features,label in sequences:
    x.append(features)
    y.append(label)

In [4]:
len_x = []
for one_seq in x:
    len_x.append(len(one_seq))
pd.Series(len_x).describe()

count    7341.000000
mean       91.684920
std        56.610559
min        35.000000
25%        57.000000
50%        69.000000
75%        90.000000
max       233.000000
dtype: float64

In [5]:
#Padding the sequence with the values in last row to max length
to_pad = 233 # follow max
new_seq = []
for one_seq in x:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(3, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
X = np.stack(new_seq)

#truncate the sequence to length xx
from keras.preprocessing import sequence
seq_len = 91 #follow mean, either round up or down to nearest int
X=sequence.pad_sequences(X, maxlen=seq_len, padding='post', dtype='float', truncating='post')

Using TensorFlow backend.


In [6]:
import pickle

pickle_out = open("X_LSTM70.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y_LSTM70.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [1]:
#breakpoint()
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Embedding
from tensorflow.keras.layers import Conv2D, MaxPooling2D ,LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from datetime import datetime
import time
import pickle


pickle_in = open("X_LSTM70.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y_LSTM70.pickle","rb")
y = pickle.load(pickle_in)

seq_len = 91
X = X *1.0/100 - 1.0 

dense_layers = [1]
layer_sizes = [128]
LSTM_layers = [2]


for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for LSTM_layer in LSTM_layers:
            NAME = "{}-LSTM-{}-nodes-{}-dense-{}".format(LSTM_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()
            
            model.add(CuDNNLSTM(layer_size, input_shape=X.shape[1:], return_sequences=True))
            
            
            for a in range(LSTM_layer-2):
                model.add(CuDNNLSTM(layer_size, input_shape=X.shape[1:], return_sequences=True))
            
            model.add(CuDNNLSTM(layer_size, input_shape=(X.shape[1:], 3)))
            
            for b in range(dense_layer):
                model.add(Dense(100, activation='relu'))
                model.add(Dropout(0.1))
            
            
            model.add(Dense(3, activation='softmax'))
            
            tensorboard = TensorBoard(log_dir="C:\\tmp\\logs\\example\\{}".format(NAME))
            #earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=2)
            #adam = tf.keras.optimizers.Adam(lr=0.01)
            
            model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
            
            print(model.summary())
            
            model.fit(X, y, batch_size=10, epochs=20, validation_split=0.1, verbose=1, callbacks=[tensorboard])
            
#model.save('AAFT70LSTM.model')
            
print("Run the command line on anaconda prompt:\n" \
          "--> tensorboard --logdir=/tmp/logs/ --host localhost --port 8088 " \
          "\nThen open http://localhost:8088/ into your web browser")

W0330 23:13:44.951806  2392 deprecation.py:506] From C:\Users\admin\Anaconda3\envs\gputest\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2-LSTM-128-nodes-1-dense-1585581224
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 91, 128)           68096     
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               132096    
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 213,395
Trainable params: 213,395
Non-trainable params: 0
_________________________________________________________________
None
Train on 6606 samples, validate on 735 samples
Epoch 1/20
6606/6606 [======

In [1]:
import os
import csv
import tensorflow as tf
import numpy as np
import pandas as pd
import numpy
from tensorflow.keras.preprocessing import sequence

path = r"E:\IDM downloads\Documents\Uni\SIT University\FYP\My FYP\New data\Original"
CATEGORIES = ["Score 1", "Score 2", "Score 3"]

def prepare(filepath):
    sequences = []
    traindata = pd.read_csv(filepath, header=None,names=['x', 'y', 'z'])
    value = traindata.values
    sequences.append((value))  # add this to our training_data
    to_pad = 233
    new_seq = []
    for one_seq in sequences:
        len_one_seq = len(one_seq)
        last_val = one_seq[-1]
        n = to_pad - len_one_seq
   
        to_concat = np.repeat(one_seq[-1], n).reshape(3, n).transpose()
        new_one_seq = np.concatenate([one_seq, to_concat])
        new_seq.append(new_one_seq)
    X = np.stack(new_seq)
    
    #truncate the sequence to length xx
    seq_len = 91
    X=sequence.pad_sequences(X, maxlen=seq_len, padding='post', dtype='float', truncating='post')
    return X *1.0/100 - 1.0 
   
    
model = tf.keras.models.load_model("AAFT70LSTM.model")

for filename in os.listdir(path):
    prediction = model.predict([prepare(os.path.join(path,filename))])
    
    print(filename)

    print(prediction[0])

    print(CATEGORIES[int(np.argmax(prediction[0]))])

    print("\n")

W0214 18:42:53.936623 16368 deprecation.py:506] From C:\Users\admin\Anaconda3\envs\gputest\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0214 18:42:53.938618 16368 deprecation.py:506] From C:\Users\admin\Anaconda3\envs\gputest\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0214 18:42:53.939616 16368 deprecation.py:506] From C:\Users\admin\Anaconda3\envs\gputest\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.pyth

P01_TS_1.csv
[1.05731466e-07 1.37668126e-03 9.98623252e-01]
Score 3


P01_TS_2.csv
[2.5467313e-09 3.5447945e-05 9.9996459e-01]
Score 3


P02_TS_1.csv
[2.1281483e-08 6.7037690e-05 9.9993289e-01]
Score 3


P02_TS_2.csv
[2.4817875e-06 3.6132801e-02 9.6386468e-01]
Score 3


P02_TS_3.csv
[1.4609939e-06 7.3261135e-03 9.9267238e-01]
Score 3


P04_TS_1.csv
[4.0871328e-08 5.1931060e-05 9.9994802e-01]
Score 3


P04_TS_2.csv
[4.5438227e-07 5.2760268e-05 9.9994683e-01]
Score 3


P04_TS_3.csv
[1.1950857e-07 1.9935846e-04 9.9980050e-01]
Score 3


P05_TS_1.csv
[4.9182969e-09 1.5361042e-04 9.9984634e-01]
Score 3


P05_TS_2.csv
[1.6110872e-07 3.5807351e-04 9.9964178e-01]
Score 3


P05_TS_3.csv
[5.791735e-09 9.426348e-05 9.999057e-01]
Score 3


P07_TS_1.csv
[2.1873193e-05 9.2989326e-01 7.0084915e-02]
Score 2


P07_TS_2.csv
[3.8808292e-05 9.8377466e-01 1.6186578e-02]
Score 2


P07_TS_3.csv
[6.7775532e-06 9.9993432e-01 5.8858826e-05]
Score 2


P08_TS_1.csv
[7.2663386e-07 2.7566475e-03 9.9724269e-01]
Score

In [2]:
prediction = model.predict([prepare('P24_TS_1_001.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P24_TS_1_002.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P24_TS_1_003.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P07_TS_1_001.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P07_TS_1_002.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P07_TS_1_003.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P01_TS_1_001.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P01_TS_1_002.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

prediction = model.predict([prepare('P01_TS_1_003.csv')])

print(prediction[0])

print(CATEGORIES[int(np.argmax(prediction[0]))])

print("\n")

[9.9997520e-01 2.0670081e-05 4.1715884e-06]
Score 1


[9.9997830e-01 1.4429069e-05 7.3298347e-06]
Score 1


[9.9981302e-01 1.8154636e-04 5.4757934e-06]
Score 1


[3.5620909e-05 9.9951649e-01 4.4784282e-04]
Score 2


[8.3453648e-05 9.9921632e-01 7.0011825e-04]
Score 2


[2.2757472e-06 9.9997723e-01 2.0492100e-05]
Score 2


[2.1013182e-05 9.1180584e-04 9.9906725e-01]
Score 3


[3.3582005e-08 1.0028026e-04 9.9989963e-01]
Score 3


[1.5154270e-06 6.5876542e-05 9.9993265e-01]
Score 3


